In [ ]:
%%capture
!pip install corus nerus razdel sklearn_crfsuite

In [ ]:
import re

from itertools import islice
from nerus import load_nerus
from razdel import tokenize
from tqdm.autonotebook import tqdm
from sklearn_crfsuite import CRF, metrics
from sklearn.model_selection import train_test_split

/tmp/ipython-input-3775069905.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
!wget https://storage.yandexcloud.net/natasha-nerus/data/nerus_lenta.conllu.gz

--2025-09-06 11:41:28--  https://storage.yandexcloud.net/natasha-nerus/data/nerus_lenta.conllu.gz
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1961465886 (1.8G) [application/octet-stream]
Saving to: ‘nerus_lenta.conllu.gz’

nerus_lenta.conllu. 100%[===================>]   1.83G  23.3MB/s    in 85s     

2025-09-06 11:42:54 (22.0 MB/s) - ‘nerus_lenta.conllu.gz’ saved [1961465886/1961465886]



### Загрузка корпуса

In [ ]:
num_docs = 2000

docs = list(
    tqdm(
        islice(load_nerus("nerus_lenta.conllu.gz"), num_docs),
        total=num_docs,
        desc="Loading Nerus"
    )
)

print(f"Loaded {len(docs)} documents")

Loading Nerus:   0%|          | 0/2000 [00:00<?, ?it/s]

Loaded 2000 documents


In [ ]:
def doc_to_data(doc):
    X, y = [], []
    for sent in doc.sents:
        tokens, labels = [], []
        for token in sent.tokens:
            tokens.append(token.text)
            labels.append(token.pos)
        X.append(tokens)
        y.append(labels)
    return X, y

In [ ]:
doc_example = doc_to_data(docs[0])

sent_example = doc_example[0][0]
labels_example = doc_example[1][0]

print("Пример предложения:", sent_example)
print("Разметка:", labels_example)

Пример предложения: ['Вице-премьер', 'по', 'социальным', 'вопросам', 'Татьяна', 'Голикова', 'рассказала', ',', 'в', 'каких', 'регионах', 'России', 'зафиксирована', 'наиболее', 'высокая', 'смертность', 'от', 'рака', ',', 'сообщает', 'РИА', 'Новости', '.']
Разметка: ['NOUN', 'ADP', 'ADJ', 'NOUN', 'PROPN', 'PROPN', 'VERB', 'PUNCT', 'ADP', 'DET', 'NOUN', 'PROPN', 'VERB', 'ADV', 'ADJ', 'NOUN', 'ADP', 'NOUN', 'PUNCT', 'VERB', 'PROPN', 'PROPN', 'PUNCT']


In [ ]:
X_all, y_all = [], []
for doc in docs:
    X, y = doc_to_data(doc)
    X_all.extend(X)
    y_all.extend(y)

### Предикторы

In [ ]:
def word_shape(word):
    return re.sub(r"[A-ZА-Я]", "X",
           re.sub(r"[a-zа-я]", "x",
           re.sub(r"[0-9]", "d", word)))

def word2features(sent, i):
    word = sent[i]
    features = {
        "bias": 1.0,
        "word.lower": word.lower(),
        "word[-3:]": word[-3:],
        "word[-2:]": word[-2:],
        "word[:2]": word[:2],
        "word[:3]": word[:3],
        "word.isupper": word.isupper(),
        "word.istitle": word.istitle(),
        "word.isdigit": word.isdigit(),
        "word.shape": word_shape(word),
    }
    if i > 0:
        prev = sent[i-1]
        features.update({
            "-1:word.lower": prev.lower(),
            "-1:word[-3:]": prev[-3:],
            "-1:word[-2:]": prev[-2:],
            "-1:word[:2]": prev[:2],
            "-1:word[:3]": prev[:3],
            "-1:word.istitle": prev.istitle(),
            "-1:word.isupper": prev.isupper(),
            "-1:word.shape": word_shape(prev),
        })
    else:
        features["BOS"] = True

    if i < len(sent)-1:
        nxt = sent[i+1]
        features.update({
            "+1:word.lower": nxt.lower(),
            "+1:word[-3:]": nxt[-3:],
            "+1:word[-2:]": nxt[-2:],
            "+1:word[:2]": nxt[:2],
            "+1:word[:3]": nxt[:3],
            "+1:word.istitle": nxt.istitle(),
            "+1:word.isupper": nxt.isupper(),
            "+1:word.shape": word_shape(nxt),
        })
    else:
        features["EOS"] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

In [ ]:
X_feats = []

for sent in tqdm(X_all, desc="Extracting features"):
    X_feats.append(sent2features(sent))

Extracting features:   0%|          | 0/23500 [00:00<?, ?it/s]

In [ ]:
X_feats[0]

[{'bias': 1.0,
  'word.lower': 'вице-премьер',
  'word[-3:]': 'ьер',
  'word[-2:]': 'ер',
  'word[:2]': 'Ви',
  'word[:3]': 'Виц',
  'word.isupper': False,
  'word.istitle': False,
  'word.isdigit': False,
  'word.shape': 'Xxxx-xxxxxxx',
  'BOS': True,
  '+1:word.lower': 'по',
  '+1:word[-3:]': 'по',
  '+1:word[-2:]': 'по',
  '+1:word[:2]': 'по',
  '+1:word[:3]': 'по',
  '+1:word.istitle': False,
  '+1:word.isupper': False,
  '+1:word.shape': 'xx'},
 {'bias': 1.0,
  'word.lower': 'по',
  'word[-3:]': 'по',
  'word[-2:]': 'по',
  'word[:2]': 'по',
  'word[:3]': 'по',
  'word.isupper': False,
  'word.istitle': False,
  'word.isdigit': False,
  'word.shape': 'xx',
  '-1:word.lower': 'вице-премьер',
  '-1:word[-3:]': 'ьер',
  '-1:word[-2:]': 'ер',
  '-1:word[:2]': 'Ви',
  '-1:word[:3]': 'Виц',
  '-1:word.istitle': False,
  '-1:word.isupper': False,
  '-1:word.shape': 'Xxxx-xxxxxxx',
  '+1:word.lower': 'социальным',
  '+1:word[-3:]': 'ным',
  '+1:word[-2:]': 'ым',
  '+1:word[:2]': 'со',
  '

### Разбиение на train и test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_feats, y_all, test_size=0.1, random_state=42)

### Обучение CRF

In [ ]:
crf = CRF(
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CRF(all_possible_transitions=True)

### Метрики на тесте

In [ ]:
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         ADJ       0.95      0.97      0.96      3735
         ADP       1.00      1.00      1.00      4974
         ADV       0.97      0.94      0.95      1048
         AUX       0.93      0.96      0.95       309
       CCONJ       0.98      1.00      0.99      1016
         DET       0.95      0.95      0.95       658
        NOUN       0.98      0.98      0.98     11745
         NUM       0.97      0.95      0.96       806
        PART       0.98      0.95      0.96       532
        PRON       0.98      0.97      0.97      1527
       PROPN       0.96      0.97      0.97      3176
       PUNCT       1.00      1.00      1.00      7631
       SCONJ       0.96      0.99      0.97       733
         SYM       0.90      1.00      0.95         9
        VERB       0.98      0.97      0.97      4938
           X       0.94      0.86      0.90       392

    accuracy                           0.98     43229
   macro avg       0.96   

### Примеры работы на омонимах

In [ ]:
def tag_sentence(text, crf_model):
    tokens = [t.text for t in tokenize(text)]
    feats = sent2features(tokens)
    tags = crf_model.predict_single(feats)
    return list(zip(tokens, tags))

In [ ]:
tag_sentence("Штирлиц открыл окно. Из окна дуло. Штирлиц закрыл окно, и дуло исчезло.", crf)

[('Штирлиц', 'NOUN'),
 ('открыл', 'VERB'),
 ('окно', 'NOUN'),
 ('.', 'PUNCT'),
 ('Из', 'ADP'),
 ('окна', 'NOUN'),
 ('дуло', 'VERB'),
 ('.', 'PUNCT'),
 ('Штирлиц', 'PROPN'),
 ('закрыл', 'VERB'),
 ('окно', 'NOUN'),
 (',', 'PUNCT'),
 ('и', 'CCONJ'),
 ('дуло', 'NOUN'),
 ('исчезло', 'VERB'),
 ('.', 'PUNCT')]

In [ ]:
tag_sentence("По стене ползет кирпич, оловянный как стекло, ну и пусть себе летит, нам не нужен пенопласт.", crf)

[('По', 'ADP'),
 ('стене', 'NOUN'),
 ('ползет', 'VERB'),
 ('кирпич', 'NOUN'),
 (',', 'PUNCT'),
 ('оловянный', 'ADJ'),
 ('как', 'SCONJ'),
 ('стекло', 'NOUN'),
 (',', 'PUNCT'),
 ('ну', 'PART'),
 ('и', 'CCONJ'),
 ('пусть', 'PART'),
 ('себе', 'PRON'),
 ('летит', 'VERB'),
 (',', 'PUNCT'),
 ('нам', 'PRON'),
 ('не', 'PART'),
 ('нужен', 'ADJ'),
 ('пенопласт', 'NOUN'),
 ('.', 'PUNCT')]

In [ ]:
tag_sentence("Необходимо организовать сбор бумаги, стекла и пластика с повторной сортировкой на свалке.", crf)

[('Необходимо', 'ADJ'),
 ('организовать', 'VERB'),
 ('сбор', 'NOUN'),
 ('бумаги', 'NOUN'),
 (',', 'PUNCT'),
 ('стекла', 'NOUN'),
 ('и', 'CCONJ'),
 ('пластика', 'NOUN'),
 ('с', 'ADP'),
 ('повторной', 'ADJ'),
 ('сортировкой', 'NOUN'),
 ('на', 'ADP'),
 ('свалке', 'NOUN'),
 ('.', 'PUNCT')]

In [ ]:
tag_sentence("Стол пошатнулся, стакан разбился, и вода моментально стекла на пол.", crf)

[('Стол', 'NOUN'),
 ('пошатнулся', 'VERB'),
 (',', 'PUNCT'),
 ('стакан', 'NOUN'),
 ('разбился', 'VERB'),
 (',', 'PUNCT'),
 ('и', 'CCONJ'),
 ('вода', 'NOUN'),
 ('моментально', 'ADV'),
 ('стекла', 'VERB'),
 ('на', 'ADP'),
 ('пол', 'NOUN'),
 ('.', 'PUNCT')]

In [ ]:
tag_sentence("Печь совсем остыла, и в доме стало холодно.", crf)

[('Печь', 'NOUN'),
 ('совсем', 'ADV'),
 ('остыла', 'VERB'),
 (',', 'PUNCT'),
 ('и', 'CCONJ'),
 ('в', 'ADP'),
 ('доме', 'NOUN'),
 ('стало', 'VERB'),
 ('холодно', 'ADJ'),
 ('.', 'PUNCT')]

In [ ]:
tag_sentence("В эту субботу мы хотим снова печь пироги с яблоками.", crf)

[('В', 'ADP'),
 ('эту', 'DET'),
 ('субботу', 'NOUN'),
 ('мы', 'PRON'),
 ('хотим', 'VERB'),
 ('снова', 'ADV'),
 ('печь', 'VERB'),
 ('пироги', 'NOUN'),
 ('с', 'ADP'),
 ('яблоками', 'NOUN'),
 ('.', 'PUNCT')]